In [2]:
#pySpark Datatype is used when schema cannot be defined using list or string
#Data types types: struct,map,array

In [9]:
import findspark

In [10]:
findspark.init()

In [11]:
from pyspark.sql import SparkSession

In [14]:
spark=SparkSession.builder.appName("RDDExample").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 04:23:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
from pyspark.sql.types import *

In [18]:
data=[(1,'a',30),(2,'b',32)]
user_schema=StructType([StructField("id",IntegerType()),
                        StructField("name",StringType()),
                        StructField("age",IntegerType())
                       ])
df=spark.createDataFrame(data,user_schema)

In [21]:
df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  1|   a| 30|
|  2|   b| 32|
+---+----+---+



In [23]:
data2= [
    (1, "Alice", ["Reading", "Hiking"]),
    (2, "Bob", ["Swimming", "Gardening", "Painting"]),
    (3, "Charlie", ["Cooking"]),
    (4, "David", ["Photography", "Skiing", "Cooking"])
]
user_schema2=StructType([StructField("id",IntegerType()),
                        StructField("name",StringType()),
                        StructField("hobbies",ArrayType(StringType()))
                       ])
df2=spark.createDataFrame(data2,user_schema2)
df2.show()

+---+-------+--------------------+
| id|   name|             hobbies|
+---+-------+--------------------+
|  1|  Alice|   [Reading, Hiking]|
|  2|    Bob|[Swimming, Garden...|
|  3|Charlie|           [Cooking]|
|  4|  David|[Photography, Ski...|
+---+-------+--------------------+



In [25]:
#get output as 1,Alice,Reading;1,Alice,hiking; 2,bob,swimming etc
from pyspark.sql.functions import *

In [27]:
df2.select("id","name",explode("hobbies")).show()

+---+-------+-----------+
| id|   name|        col|
+---+-------+-----------+
|  1|  Alice|    Reading|
|  1|  Alice|     Hiking|
|  2|    Bob|   Swimming|
|  2|    Bob|  Gardening|
|  2|    Bob|   Painting|
|  3|Charlie|    Cooking|
|  4|  David|Photography|
|  4|  David|     Skiing|
|  4|  David|    Cooking|
+---+-------+-----------+



In [28]:
df2.withColumn("newhobby",explode("hobbies")).show()#creates new derived column


+---+-------+--------------------+-----------+
| id|   name|             hobbies|   newhobby|
+---+-------+--------------------+-----------+
|  1|  Alice|   [Reading, Hiking]|    Reading|
|  1|  Alice|   [Reading, Hiking]|     Hiking|
|  2|    Bob|[Swimming, Garden...|   Swimming|
|  2|    Bob|[Swimming, Garden...|  Gardening|
|  2|    Bob|[Swimming, Garden...|   Painting|
|  3|Charlie|           [Cooking]|    Cooking|
|  4|  David|[Photography, Ski...|Photography|
|  4|  David|[Photography, Ski...|     Skiing|
|  4|  David|[Photography, Ski...|    Cooking|
+---+-------+--------------------+-----------+



In [31]:
df2.withColumn("hobbies",explode("hobbies")).show()#replaces existing

+---+-------+-----------+
| id|   name|    hobbies|
+---+-------+-----------+
|  1|  Alice|    Reading|
|  1|  Alice|     Hiking|
|  2|    Bob|   Swimming|
|  2|    Bob|  Gardening|
|  2|    Bob|   Painting|
|  3|Charlie|    Cooking|
|  4|  David|Photography|
|  4|  David|     Skiing|
|  4|  David|    Cooking|
+---+-------+-----------+



In [34]:
df3=df2.withColumn("ingestion_date",current_timestamp()).show(truncate=False)#try truncate=True ans see what happens

+---+-------+-------------------------------+-----------------------+
|id |name   |hobbies                        |ingestion_date         |
+---+-------+-------------------------------+-----------------------+
|1  |Alice  |[Reading, Hiking]              |2023-09-22 04:49:24.115|
|2  |Bob    |[Swimming, Gardening, Painting]|2023-09-22 04:49:24.115|
|3  |Charlie|[Cooking]                      |2023-09-22 04:49:24.115|
|4  |David  |[Photography, Skiing, Cooking] |2023-09-22 04:49:24.115|
+---+-------+-------------------------------+-----------------------+

